In [1]:
import pandas as pd
import geopandas as gpd
import movingpandas as mpd
import numpy as np
from datetime import timedelta, datetime
from ast import literal_eval
import folium
import time
import warnings
import pickle
import sys

warnings.filterwarnings('ignore')

print("Geopandas has version {}".format(gpd.__version__))
print("Movingpandas has version {}".format(mpd.__version__))

# add paths for modules
sys.path.append('../models')
sys.path.append('../visualization')
sys.path.append('../features')
# import modules
import  visualize
import geometry_utils
from maritime_traffic_network import MaritimeTrafficNetwork

Geopandas has version 0.13.2
Movingpandas has version 0.17.1


In [2]:
# load network from pickle
model_date = '202204'
location = 'tromso'
DP_tol = 10
min_samples = 13
data_size = 'full'
refined = ''

model_name = model_date+'_waypoints_DP'+str(DP_tol)+'_HDBSCAN'+str(min_samples)+'_'+location+'_'+data_size+'_UTM'+refined
model_path = '../../models/networks/best_networks/' + model_name + '.obj'
fileObj = open(model_path, 'rb')
network = pickle.load(fileObj)
fileObj.close()
network.hyperparameters

{'Data': '../../data/processed/202204_points_tromso_cleaned_meta_full_dualSplit_2.parquet',
 'DP_tolerance': 10,
 'clustering_method': 'HDBSCAN',
 'clustering_metric': 'mahalanobis',
 'clustering_min_samples': 13,
 'clustering_min_cluster_size': 13,
 'clustering_eps': 0,
 'clustering_metric_V': array([[1.  , 0.  , 0.  , 0.  , 0.  ],
        [0.  , 1.  , 0.  , 0.  , 0.  ],
        [0.  , 0.  , 0.01, 0.  , 0.  ],
        [0.  , 0.  , 0.  , 0.01, 0.  ],
        [0.  , 0.  , 0.  , 0.  , 1.  ]]),
 'graph_generation_max_distance': 20,
 'graph_generation_max_angle': 45}

In [3]:
G = network.G.copy()
# Convert the graph to the desired format
converted_data = {}
for node, data in G.nodes(data=True):
    neighbors = list(G.successors(node))
    position = data.get('position', None)
    converted_data[node] = (position, neighbors)



In [4]:
from leuvenmapmatching.matcher.distance import DistanceMatcher
from leuvenmapmatching.map.inmem import InMemMap

ll_map = InMemMap('MTM', graph=converted_data, use_latlon=True, crs_lonlat='EPSG:4326')
xy_map = ll_map.to_xy()

In [5]:
trajectories = network.trajectories
paths = {}
j=0
for trajectory in trajectories:
    coord_list = []
    traj = trajectory.to_point_gdf()
    for i in range(0, len(traj)):
        lon = traj['lon'].iloc[i]
        lat = traj['lat'].iloc[i]
        coord_list.append(tuple([lon, lat]))
    paths[j] = coord_list
    j+=1

In [6]:
path = paths[2]
matcher = DistanceMatcher(ll_map, max_dist=1000, obs_noise=1000, min_prob_norm=0.01, max_lattice_width=50000)
states, _ = matcher.match(path)
nodes = matcher.path_pred_onlynodes

print("States\n------")
print(states)
print("Nodes\n------")
print(nodes)
print("")
matcher.print_lattice_stats()

Searching closeby nodes with linear search, use an index and set max_dist


States
------
[(130, 129), (130, 129), (130, 129), (130, 129), (130, 129), (130, 129), (130, 129), (130, 129), (130, 129), (130, 129), (130, 129), (129, 128), (129, 128), (129, 128), (129, 128), (129, 128), (129, 128), (129, 128), (129, 128), (129, 128), (129, 128), (129, 128), (129, 128), (129, 128), (129, 128), (129, 128), (129, 128), (129, 128), (129, 128), (129, 128), (129, 128), (129, 128), (129, 128), (129, 128), (129, 128), (129, 128), (129, 128), (129, 128), (129, 128), (128, 4), (128, 4), (128, 4), (128, 4), (128, 4), (128, 4), (128, 4), (128, 4), (128, 4), (128, 4), (128, 4), (128, 4), (128, 4), (4, 353), (4, 353), (4, 353), (4, 353), (4, 353), (4, 353), (4, 353), (4, 353), (4, 353), (4, 353), (4, 353), (4, 353), (4, 353), (4, 353), (4, 353), (353, 350), (353, 350), (353, 350), (353, 350), (353, 350), (353, 350), (353, 350), (353, 350), (353, 350), (353, 350), (353, 350), (353, 350), (350, 349), (350, 349), (350, 349), (350, 349), (350, 349), (349, 323), (349, 323), (349, 323

In [16]:
# plot network and basemap
#network.prune_graph(5, 'refined')
map = network.map_graph(pruned=True, line_weight=2, min_passages=3, location=location)

# add some trajectories
selection = [1]
mmsis = network.gdf.mmsi.unique()[selection]
for mmsi in mmsis:
    print(mmsi)
    trajectory = trajectories.get_trajectory(mmsi)
    path_df_sspd2, evaluation_results_sspd2 = network.trajectory_to_path_sspd2(trajectory, verbose=False)
    print(evaluation_results_sspd2[['message', 'SSPD']])
    path_df_sspd, evaluation_results_sspd = network.trajectory_to_path_sspd(trajectory, verbose=False)
    print(evaluation_results_sspd[['message', 'SSPD']])
    print('------------------')
    #try:
    map = trajectory.to_line_gdf()[['geometry', 'mmsi', 'skipsgruppe']].explore(m=map, style_kwds={'weight':3, 'color':'black', 'opacity':1}, 
                                                                  name=mmsi+' trajectory')
    map = path_df_sspd.explore(m=map, name=mmsi+' edge_sequence sspd', style_kwds={'weight':3, 'color':'cyan', 'opacity':1})
    map = path_df_sspd2.explore(m=map, name=mmsi+' edge_sequence sspd2', style_kwds={'weight':3, 'color':'yellow', 'opacity':1})
    #map = geometry_utils.get_geo_df(nodes, network.waypoint_connections).explore(m=map, name=mmsi+' edge_sequence mapmatched', style_kwds={'weight':3, 'color':'pink', 'opacity':1})
    #except:
        #print(mmsi+': no path found')
        #map = geometry_utils.get_geo_df(nodes, network.waypoint_connections).explore(m=map, name=mmsi+' edge_sequence mapmatched', style_kwds={'weight':3, 'color':'yellow', 'opacity':1})
bbox = visualize.get_bounding_box(network.gdf)
map = bbox.explore(m=map, color='red', name='bounds')
folium.LayerControl().add_to(map)
map

211211730_1_2022-04-01 09:12:01
         message  SSPD
0  no_intersects   NaN
         message  SSPD
0  no_intersects   NaN
------------------


AttributeError: 'DataFrame' object has no attribute 'explore'

# 